<a href="https://colab.research.google.com/github/shahzad-r1zv1/LocalLLM_experiments/blob/main/GithubTalkie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ============================================
# Federated Multi-Repo Chat  — Enhanced, Language-Independent RAG
# --------------------------------------------
# What this notebook does:
# 1) Clone a curated list of GitHub repos into Colab
# 2) Collect relevant code/docs/config files (language independent)
# 3) Build per-repo maps (tree + entrypoint hints) for fast orientation
# 4) Chunk files (Tree-sitter when possible, fallback chunking otherwise)
# 5) Build a single *federated* hybrid search index across all repos:
# - BM25 (lexical) helps with exact identifiers/symbols
# - Embeddings + FAISS (semantic) helps with meaning & paraphrases
# 6) Provide tools for repo exploration (open_file, list_tree, search_symbol, find_references)
# 7) Ask questions either scoped to one repo or across all repos (repo='all')
# 8) Apply simple confidence guardrails to reduce hallucinations
#
#
#
# ============================================
# Why this approach works:
# - Hybrid retrieval reduces the classic RAG failure mode where embeddings miss exact names.
# - Tree-sitter chunking tends to keep functions/classes intact, improving answer quality.
# - Federated indexing lets you find patterns across repos and compare implementations.
# ============================================

In [ ]:
# ==========================================================
# CELL 0 — Install dependencies
# ----------------------------------------------------------
# gitpython: clone repos
# tree_sitter + tree_sitter_languages: syntax-aware chunking
# rank-bm25: lexical retrieval (identifiers, file names, exact tokens)
# sentence-transformers: embeddings
# faiss: fast vector search
# transformers/bitsandbytes: load an open-source LLM in Colab (4-bit)
# ==========================================================
!pip -q install gitpython faiss-cpu sentence-transformers rank-bm25 transformers accelerate bitsandbytes
!pip -q install tree_sitter tree_sitter_languages

In [ ]:
# ==========================================================
# CELL 1 — Configure curated repo list + clone
# ----------------------------------------------------------
# You control the list. Each repo gets its own folder under ROOT_DIR.
# Using a curated list keeps noise down and makes cross-repo linking meaningful.
# ==========================================================
import os, re, glob, shutil
from pathlib import Path

from git import Repo

REPOS = [
    {"name": "repoA", "url": "https://github.com/droidrun/droidrun"},
    {"name": "repoB", "url": "https://github.com/mvysny/karibu-testing"},
    # {"name": "repoC", "url": "https://github.com/OWNER_C/REPO_C"},
]

ROOT_DIR = "/content/repos"

if os.path.exists(ROOT_DIR):
    shutil.rmtree(ROOT_DIR)
os.makedirs(ROOT_DIR, exist_ok=True)

repo_dirs = {}  # repo_name -> local path

for r in REPOS:
    name, url = r["name"], r["url"]
    dest = os.path.join(ROOT_DIR, name)
    Repo.clone_from(url, dest)
    repo_dirs[name] = dest
    print(f"Cloned {name}: {url}")

print("\nRepos ready:", list(repo_dirs.keys()))


In [ ]:


# ==========================================================
# CELL 2 — Collect files (language independent)
# ----------------------------------------------------------
# Goal: include source + docs + common config files; exclude build/vendor/cache.
# This keeps indexing cost down and improves retrieval quality.
# ==========================================================
INCLUDE_EXT = {
    ".py", ".js", ".ts", ".jsx", ".tsx", ".java", ".go", ".rs",
    ".cpp", ".c", ".h", ".hpp", ".cs", ".php", ".rb", ".swift", ".kt", ".scala",
    ".sql", ".md", ".rst", ".txt", ".yaml", ".yml", ".toml", ".ini", ".json", ".xml",
    ".gradle", ".properties", ".cfg", ".env", ".sh", ".ps1"
}

EXCLUDE_DIRS = {
    ".git", "node_modules", "dist", "build", "target", ".venv", "venv", "__pycache__",
    ".next", ".cache", ".idea", ".vscode", "coverage", ".pytest_cache", ".mypy_cache",
    ".gradle", ".terraform", ".npm", ".yarn", ".pnpm-store", ".cargo", ".tox"
}

MAX_FILE_BYTES = 1_200_000  # skip files > 1.2MB (tune for your repos)


def is_excluded(path: str) -> bool:
    parts = set(Path(path).parts)
    return len(parts.intersection(EXCLUDE_DIRS)) > 0


def collect_files(root: str):
    out = []
    for p in glob.glob(root + "/**/*", recursive=True):
        # Early continues reduce indentation bugs and keep logic clear.
        if not os.path.isfile(p):
            continue
        if is_excluded(p):
            continue

        path = Path(p)
        ext = path.suffix.lower()
        name = path.name.lower()

        # Include known extensions plus special filenames (Makefile, Dockerfile)
        if ext in INCLUDE_EXT or name in {"makefile", "dockerfile"}:
            if os.path.getsize(p) <= MAX_FILE_BYTES:
                out.append(p)

    return sorted(out)


repo_files = {name: collect_files(path) for name, path in repo_dirs.items()}
for name, flist in repo_files.items():
    print(name, "files:", len(flist))



In [ ]:

# ==========================================================
# CELL 3 — Build per-repo maps (tree + entrypoint hints)
# ----------------------------------------------------------
# Repo maps act like a "table of contents" for the model.
# This reduces flailing and gives consistent orientation across repos.
# ==========================================================
ENTRYPOINT_HINTS = [
    r"^main\.(py|js|ts|go|rs|java|kt|cs)$",
    r"^app\.(py|js|ts)$",
    r"^index\.(js|ts)$",
    r"^server\.(js|ts|py)$",
    r"^manage\.py$",
    r"^pom\.xml$",
    r"^build\.gradle(\.kts)?$",
    r"^package\.json$",
    r"^Dockerfile$",
    r"^Makefile$",
]


def repo_tree(root: str, max_lines: int = 300) -> str:
    lines = []
    count = 0
    rootp = Path(root)
    for p in sorted(rootp.rglob("*")):
        if count >= max_lines:
            lines.append("... (tree truncated)")
            break
        if is_excluded(str(p)) or p.is_dir():
            continue
        lines.append(str(p).replace(root + "/", ""))
        count += 1
    return "\n".join(lines)


def find_entrypoints(files_list, repo_root: str):
    hits = []
    for fp in files_list:
        rel = fp.replace(repo_root + "/", "")
        base = Path(rel).name
        for pat in ENTRYPOINT_HINTS:
            if re.match(pat, base, flags=re.IGNORECASE):
                hits.append(rel)
                break
    return hits[:25]


REPO_MAPS = {}
for name, root in repo_dirs.items():
    tree = repo_tree(root)
    eps = find_entrypoints(repo_files[name], root)

    REPO_MAPS[name] = (
        f"REPO: {name}\n"
        f"REPO TREE (partial):\n{tree}\n\n"
        f"LIKELY ENTRYPOINTS / IMPORTANT FILES:\n- "
        + "\n- ".join(eps if eps else ["(none detected)"])
    )

print(REPO_MAPS[list(REPO_MAPS.keys())[0]][:1600])




In [ ]:
# ==========================================================
# CELL 4 — Chunking (Tree-sitter when possible)
# ----------------------------------------------------------
# Why chunking matters:
# - RAG retrieves *chunks*, not whole repos.
# - Better chunks = better context = better answers.
# Tree-sitter helps preserve natural boundaries (functions/classes).
# Fallback chunking handles everything else safely.
# ==========================================================
from tree_sitter_languages import get_parser

TS_LANG = {
    ".py": "python",
    ".js": "javascript",
    ".ts": "typescript",
    ".jsx": "javascript",
    ".tsx": "tsx",
    ".java": "java",
    ".go": "go",
    ".rs": "rust",
    ".c": "c",
    ".h": "c",
    ".cpp": "cpp",
    ".hpp": "cpp",
    ".cs": "c_sharp",
    ".php": "php",
    ".rb": "ruby",
    ".swift": "swift",
    ".kt": "kotlin",
    ".scala": "scala",
}


def read_text(path: str) -> str:
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        return f.read()


def fallback_chunk(text: str, chunk_size: int = 1400, overlap: int = 200):
    chunks = []
    i = 0
    while i < len(text):
        chunks.append(text[i : i + chunk_size])
        i += chunk_size - overlap
    return chunks


def ts_chunk(code: str, ext: str, max_chars: int = 1600):
    lang = TS_LANG.get(ext)
    if not lang:
        return None
    try:
        parser = get_parser(lang)
        tree = parser.parse(bytes(code, "utf8"))
        root = tree.root_node

        chunks, buf = [], ""
        for child in root.children:
            seg = code[child.start_byte : child.end_byte]
            if not seg.strip():
                continue

            if len(buf) + len(seg) > max_chars and buf.strip():
                chunks.append(buf)
                buf = seg
            else:
                buf += ("\n" if buf else "") + seg

        if buf.strip():
            chunks.append(buf)

        return chunks if chunks else None
    except Exception:
        return None


def chunk_file(path: str):
    code = read_text(path)
    if len(code.strip()) < 20:
        return []

    ext = Path(path).suffix.lower()
    chunks = ts_chunk(code, ext)
    if chunks is None:
        chunks = fallback_chunk(code)

    return chunks




In [ ]:
# ==========================================================
# CELL 5 — Build federated docs/metas across ALL repos
# ----------------------------------------------------------
# We create one global list of chunks and metadata:
#   metas_all[i] tells you which repo/file/chunk docs_all[i] came from.
# This is the key to federated search.
# ==========================================================

docs_all, metas_all = [], []

for repo_name, root in repo_dirs.items():
    for fp in repo_files[repo_name]:
        rel = fp.replace(root + "/", "")
        chunks = chunk_file(fp)
        for i, ch in enumerate(chunks):
            docs_all.append(ch)
            metas_all.append({"repo": repo_name, "file": rel, "chunk": i})

print("Total federated chunks:", len(docs_all))
print("Example meta:", metas_all[0] if metas_all else None)



In [ ]:

# ==========================================================
# CELL 6 — Hybrid retrieval index (BM25 + Embeddings + FAISS)
# ----------------------------------------------------------
# BM25: best for exact strings (AuthService, route names, filenames, flags)
# Embeddings: best for meaning ("how does auth work", paraphrases, intent)
# We merge both candidate pools and score them into a final ranking.
# ==========================================================
import numpy as np
import faiss
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer


def tokenize_for_bm25(text: str):
    # Language-agnostic-ish tokenization:
    # identifiers, numbers, and a few common operators/delimiters
    return re.findall(r"[A-Za-z_][A-Za-z0-9_]{1,}|\d+|==|!=|<=|>=|->|::|\.", text)


bm25_corpus = [tokenize_for_bm25(d) for d in docs_all]
bm25 = BM25Okapi(bm25_corpus)

embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
emb = embed_model.encode(docs_all, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)

faiss_index = faiss.IndexFlatIP(emb.shape[1])  # cosine similarity via normalized vectors
faiss_index.add(emb)


def hybrid_retrieve(query: str, k: int = 12, pool: int = 60, alpha: float = 0.55, repo: str = "all"):
    """Hybrid retrieval across multiple repos.

    Args:
      query: the question
      k: number of final chunks returned
      pool: candidate pool from each retrieval method
      alpha: weight toward semantic (embeddings) vs lexical (BM25)
      repo: 'all' for federated or a specific repo name
    """
    q_tok = tokenize_for_bm25(query)
    bm25_scores = bm25.get_scores(q_tok)
    bm25_top = np.argsort(bm25_scores)[::-1][:pool]

    q_vec = embed_model.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    sem_scores, sem_ids = faiss_index.search(q_vec, pool)

    candidates = set(bm25_top.tolist()) | set(sem_ids[0].tolist())

    bm25_max = float(np.max(bm25_scores)) if len(bm25_scores) else 1.0
    sem_map = {int(i): float(s) for i, s in zip(sem_ids[0], sem_scores[0])}

    scored = []
    for i in candidates:
        m = metas_all[i]
        if repo != "all" and m["repo"] != repo:
            continue

        b = float(bm25_scores[i]) / (bm25_max if bm25_max > 0 else 1.0)
        s = sem_map.get(int(i), 0.0)
        score = (1 - alpha) * b + alpha * s
        scored.append((score, int(i), b, s))

    scored.sort(reverse=True, key=lambda x: x[0])

    results = []
    for score, idx, b, s in scored[:k]:
        results.append(
            {
                "score": float(score),
                "bm25": float(b),
                "sem": float(s),
                "text": docs_all[idx],
                "meta": metas_all[idx],
                "id": idx,
            }
        )

    return results



In [ ]:

# ==========================================================
# CELL 7 — Multi-repo browsing tools
# ----------------------------------------------------------
# These tools let you drive the exploration like an engineer:
# - open_file for precise line-level citations
# - list_tree to browse
# - search_symbol and find_references for quick navigation
# ==========================================================

def list_repos():
    return list(repo_dirs.keys())


def list_tree(repo: str, prefix: str = "", limit: int = 200):
    root = repo_dirs[repo]
    prefix = prefix.strip().lstrip("/")

    matches = []
    for f in repo_files[repo]:
        rel = f.replace(root + "/", "")
        if rel.startswith(prefix):
            matches.append(rel)
    return matches[:limit]


def open_file(repo: str, path: str, start: int = 1, end: int = 200):
    root = repo_dirs[repo]
    full = os.path.join(root, path)
    if not os.path.exists(full):
        return f"File not found: {repo}/{path}"

    lines = read_text(full).splitlines()
    start = max(1, start)
    end = min(len(lines), end)

    return "\n".join(f"{i+1:4d} | {lines[i]}" for i in range(start - 1, end))


def search_symbol(repo: str, symbol: str, limit: int = 50):
    root = repo_dirs[repo]
    pat = re.compile(rf"\b{re.escape(symbol)}\b")

    hits = []
    for f in repo_files[repo]:
        rel = f.replace(root + "/", "")
        if pat.search(read_text(f)):
            hits.append(rel)
            if len(hits) >= limit:
                break

    return hits


def find_references(repo: str, symbol: str, limit: int = 60):
    root = repo_dirs[repo]
    pat = re.compile(rf"\b{re.escape(symbol)}\b")

    refs = []
    for f in repo_files[repo]:
        rel = f.replace(root + "/", "")
        for i, line in enumerate(read_text(f).splitlines()):
            if pat.search(line):
                refs.append((rel, i + 1, line.strip()[:240]))
                if len(refs) >= limit:
                    return refs

    return refs


print("Repos:", list_repos())



In [ ]:

# ==========================================================
# CELL 8 — Load LLM (runs in Colab)
# ----------------------------------------------------------
# Use a reasonably capable instruct model.
# - Mistral 7B 4-bit is often OK on a Colab T4/A10
# - If you hit memory issues, switch to TinyLlama.
# ==========================================================
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"
# If VRAM issues:
# MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"


tok = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    load_in_4bit=True,
)

gen = pipeline("text-generation", model=model, tokenizer=tok)
print("Loaded:", MODEL_NAME)



In [ ]:

# ==========================================================
# CELL 9 — Federated Q&A with confidence guardrails
# ----------------------------------------------------------
# The model must cite evidence.
# If confidence is low, it asks follow-ups and suggests which files to open.
# This reduces hallucinations and encourages inspectability.
# ==========================================================
SYSTEM = """You are a senior software engineer. You can search across multiple repositories.
Rules:
- Use ONLY the provided CONTEXT + REPO MAPS + TOOL OUTPUTS for factual claims.
- Cite repo/file paths in evidence bullets.
- If context is insufficient, say what to inspect next and ask 1-2 targeted questions.
"""


def confidence_from_hits(hits):
    # Simple heuristic: top score + how quickly the relevance drops.
    if not hits:
        return 0.0
    top = hits[0]["score"]
    third = hits[2]["score"] if len(hits) > 2 else hits[-1]["score"]
    gap = top - third
    conf = max(0.0, min(1.0, top * 0.9 + (1.0 - min(1.0, gap * 2)) * 0.1))
    return float(conf)


def build_context(question: str, repo: str, k: int):
    hits = hybrid_retrieve(question, repo=repo, k=k)
    blocks = []

    for h in hits:
        m = h["meta"]
        blocks.append(
            f"[{m['repo']}/{m['file']} :: chunk {m['chunk']} :: score {h['score']:.3f} "
            f"(bm25 {h['bm25']:.2f}, sem {h['sem']:.2f})]\n"
            f"{h['text']}"
        )

    return hits, "\n\n---\n\n".join(blocks)


def ask_repo(question: str, repo: str = "all", k: int = 12, max_new_tokens: int = 550):
    """Ask a question scoped to a repo or federated across all repos.

    repo:
      - 'all' searches everything
      - or a specific repo name (e.g., 'repoA')
    """
    hits, context = build_context(question, repo=repo, k=k)
    conf = confidence_from_hits(hits)

    # Provide the model a map. For federated, include all maps.
    if repo == "all":
        maps = "\n\n".join(REPO_MAPS[r] for r in REPO_MAPS.keys())
    else:
        maps = REPO_MAPS.get(repo, f"REPO: {repo} (map not found)")

    prompt = f"""{SYSTEM}

REPO MAPS:
{maps}

QUESTION:
{question}

CONTEXT (top {k}, repo={repo}):
{context}

CONFIDENCE:
{conf:.2f}

ANSWER FORMAT:
- Direct answer
- Evidence bullets (repo/file paths)
- If confidence < 0.55: ask 1-2 clarifying questions + suggest next files/tools to inspect
"""

    out = gen(
        prompt,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.2,
    )[0]["generated_text"]

    return out[len(prompt) :].strip()



In [ ]:

# ==========================================================
# CELL 10 — Examples (run when ready)
# ----------------------------------------------------------
# 1) Federated question across all repos
# print(ask_repo("Which repo implements authentication, and where?", repo="all"))

#
# 2) Repo-scoped question
# print(ask_repo("Where is the main entrypoint and how does the app start?", repo="repoA"))
#
# 3) Use tools for precise investigation
# print(search_symbol("repoA", "AuthService"))
# print(open_file("repoA", "README.md", 1, 120))
# print(find_references("repoB", "TODO"))

# moonshot
print(ask_repo("What is the best functionality I can come up with if you combine these repos and use a common functionality for good of Testing?", repo="all"))
# ==========================================================
